#### 1. Importing Required Libraries

In [42]:
import json



from qdrant_client import QdrantClient, models
from langchain_ollama import ChatOllama



#### 2. Building Knowledge Base

In [2]:
# Qdrant client
qd_client = QdrantClient("http://localhost:6333")

In [3]:
# Embedding model configuration
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [5]:
collection_name = "zoomcamp-faq"

# Creating the collection
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE,
    )
)

True

### Loading the documents

In [13]:
with open("../documents.json", "r") as json_file:
    documents_raw = json.load(json_file)

In [14]:
# Flattening the documents
# Each document is associated with a course, so we need to extract that information
documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [15]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [16]:
points = []

for i, doc in enumerate(documents):
    # content is a concatenation of question and text
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [17]:
points[:5]

[PointStruct(id=0, vector=Document(text="Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with an

In [18]:
# Upserting the points into the collection
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:
query = "I just discovered the course. Can I still join it?"

In [24]:
search_results = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=query,
        model=model_handle
    ),
    limit=5,
    with_payload=True,
)

In [25]:
results = [] 

for point in search_results.points:
    results.append(point.payload)

In [26]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

### vector search with filter

#### create Index 

In [33]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword",
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [34]:
query = "I discovered the course. Can I still join it?"

course = "data-engineering-zoomcamp"

In [35]:
search_results = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=query,
        model=model_handle
    ),
    query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
    limit=5,
    with_payload=True,
)

In [36]:
results = [] 

for point in search_results.points:
    results.append(point.payload)


results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

vector Search

In [65]:
def vector_search(question: str) -> list:
    print('vector_search is used')
    
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [66]:
vector_search("I just discovered the course. Can I still join it?")

vector_search is used


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

### RAG with vector search

##### LLM Model 

In [59]:
def llm(prompt: str) -> str:
    chat_llm = ChatOllama(model="gemma3:4b", temperature=0.1)
    response = chat_llm.invoke(prompt)
    return response.content

In [60]:
llm("Can I still join the course?")

'Please tell me more about the course you\'re asking about! I need some information to answer your question. Specifically, I need to know:\n\n*   **What is the name of the course?** (e.g., "Introduction to Python Programming," "Watercolor Painting Basics," "Advanced Calculus")\n*   **Where is the course offered?** (e.g., "University of California, Berkeley," "Coursera," "Local Community College")\n*   **What is the deadline for enrollment?** (If you know it)\n*   **What kind of course is it?** (e.g., online, in-person, workshop, etc.)\n\nOnce you provide me with this information, I can try to find out if you can still join and what the next steps might be.'

#### Building Prompt

In [61]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [67]:
def rag(query: str) -> str:
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    response = llm(prompt)
    return response

In [68]:
rag("I just discovered the course. Can I still join it?")

vector_search is used


"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don’t leave everything for the last minute."

In [69]:
rag("How do I run kafka?")

vector_search is used


'I am sorry, but the provided context does not contain information on how to run Kafka. It discusses troubleshooting issues with Kafka scripts, docker containers, and a RisingWave workshop.'